# PySimpleGUI Timer
- PySimpleGUI を使ってタイマーウィジェットを作成
- ポモドーロタイマー(25分,5分)を作る
- Clock と合体させる

## 課題
- カウントダウンはできた
- 設定時間後に休憩時間のカウントダウンについて未実装(済)
- STOP 後に再実行すると BREAK_TIME の文字になってしまう(済)
- adjust_time が STOP するたびに引かれるため、タイマーがずれる
- 別案でおおむね完成したのでもう少し見直したい

[countdown-timer](https://docs.pysimplegui.com/en/latest/cookbook/ecookbook/tools_utilities/countdown-timer/)

[Python タイマーを作ってみた GUI編](https://fuji-pocketbook.net/timer-gui/)

In [1]:
# import PySimpleGUI as sg


# layout = [
#     [sg.Text('タイマーをセットします', size=(None, 2))],
#     [sg.Spin([m for m in range(61)]), sg.T('分'), sg.Spin([s for s in range(60)]), sg.T('秒')],
#     [sg.VPush()],
#     # [sg.Checkbox('デフォルトとして設定する', key='-DEFAULT-')],
#     [sg.OK(), sg.Cancel()]]
    

# window = sg.Window('セットタイマー', layout)

# event, values = window.read()
# if event == sg.WIN_CLOSED or event == 'Cancel':
#     user = (25, 0)
# elif event == 'OK':
#     user  = (values[0], values[1])
#     # if values['-DEFAULT-']:
#     #     with open(path, 'w', encoding='utf-8') as f:
#     #         f.write(f'{values[0]} {values[1]}')

# window.close()

In [2]:
import time
import PySimpleGUI as sg

FOCUS_M, FOCUS_S = (0, 25)
BREAK_M, BREAK_S = (0, 5)
TOTAL_M, TOTAL_S = (FOCUS_M+BREAK_M, FOCUS_S+BREAK_S)
current_m, current_s = TOTAL_M, TOTAL_S


layout = [
    [sg.Button(f'{FOCUS_M:02} : {FOCUS_S:02}', font=('impact', 35), key='-TIMER_BTN-')],
    [sg.Text(f'FOCUS TIME', font=('impact', 15), key='-MSG-'), sg.Button('Exit')],
    ]

window = sg.Window('Pomodoro Timer', layout, location=(200, 200), element_justification='center')


count_down = False
while True:
    event, values = window.read(timeout=100)

    # event manager
    if event == sg.WIN_CLOSED or event == 'Exit':
        break

    # if count_down and event == '-TIMER_BTN-':
    #         count_down = False
    #         msg = 'STOP'
    #         window['-MSG-'].update(f'{msg}')
    # elif not count_down and event == '-TIMER_BTN-':
    #         count_down = True
    #         msg = 'FOCUS'
    #         end_time = time.time() + current_m * 60 + current_s

    if event == '-TIMER_BTN-':
        if count_down:
            count_down = False
            msg = 'STOP'
            window['-MSG-'].update(f'{msg}')
        elif not count_down:
            count_down = True
            msg = 'FOCUS'
            end_time = time.time() + current_m * 60 + current_s


    # display manager
    if count_down:
        
        remaining_time = end_time - time.time()
        adjust_time = BREAK_M * 60 + BREAK_S

        if remaining_time > adjust_time:
            current_m, current_s = divmod(remaining_time-adjust_time, 60)
        elif 0 <= remaining_time <= adjust_time:
            msg = 'BREAK TIME'
            current_m, current_s = divmod(remaining_time, 60)
        elif remaining_time < 0:
            count_down = False
            msg = 'FINISHED'
            current_m, current_s = FOCUS_M, FOCUS_S
        
        window['-TIMER_BTN-'].update(f'{int(current_m):02} : {int(current_s):02}')
        window['-MSG-'].update(f'{msg}')

window.close()

# time.sleep() をつかう
- [【Python】ポモドーロタイマー：コンソールに作業と休憩時間を入力する](https://anotools.com/python/2945/)

In [ ]:
import datetime as dt
import time


FOCUS_M, FOCUS_S = (0, 3)
BREAK_M, BREAK_S = (0, 2)

today = dt.datetime.today()
focus_time = dt.datetime.combine(today, dt.time(0, FOCUS_M, FOCUS_S))
break_time = dt.datetime.combine(today, dt.time(0, BREAK_M, BREAK_S))

focus_start = True
while True:
    if focus_start:
        focus_time_str = focus_time.strftime("%M:%S")
        if focus_time_str == "59:59":
            focus_start = False
        else:
            display_text = f'FOCUS : {focus_time_str}'
            print(display_text)
            focus_time -= dt.timedelta(seconds=1)
            time.sleep(1)
    else:
        break_time_str = break_time.strftime("%M:%S")
        if break_time_str == "59:59":
            break
        else:
            display_text = f'BREAK : {break_time_str}'
            print(display_text)
            break_time -= dt.timedelta(seconds=1)
            time.sleep(1)



In [10]:
import PySimpleGUI as sg
import datetime as dt
import time


FOCUS_M, FOCUS_S = (0, 3)
BREAK_M, BREAK_S = (0, 2)

today = dt.datetime.today()
focus_time = dt.datetime.combine(today, dt.time(0, FOCUS_M, FOCUS_S))
focus_time_str = focus_time.strftime("%M:%S")
break_time = dt.datetime.combine(today, dt.time(0, BREAK_M, BREAK_S))

layout = [
    [sg.Button(f'{focus_time_str}', font=('impact', 35), key='-TIMER_BTN-')],
    [sg.Text(f'FOCUS TIME', font=('impact', 15), key='-MSG-'), sg.Button('Exit')],
    ]

window = sg.Window('Pomodoro Timer', layout, location=(200, 200), element_justification='center')


timer_start = False
focus_start = True

while True:
    event, values = window.read(timeout=100)
    if event == sg.WIN_CLOSED or event == 'Exit':
        break

    if event == '-TIMER_BTN-':
        timer_start = True
    

    if timer_start:
        if focus_start:
            focus_time_str = focus_time.strftime("%M:%S")
            if focus_time_str == "59:59":
                focus_start = False
            else:
                # display_text = f'FOCUS : {focus_time_str}'
                # print(display_text)
                window['-TIMER_BTN-'].update(focus_time_str)
                window['-MSG-'].update('FOCUS TIME')
                focus_time -= dt.timedelta(seconds=1)
                time.sleep(1)
        else:
            break_time_str = break_time.strftime("%M:%S")
            if break_time_str == "59:59":
                break
            else:
                # display_text = f'BREAK : {break_time_str}'
                # print(display_text)
                window['-TIMER_BTN-'].update(break_time_str)
                window['-MSG-'].update('BREAK TIME')
                break_time -= dt.timedelta(seconds=1)
                time.sleep(1)


window.close()